In [ ]:
%pip install - U pip setuptools wheel
%pip install tqdm
%pip install torch torchvision torchaudio
!export CUDA_PATH = "/opt/nvidia/cuda"
%pip install - U spacy[cuda11X, transformers]
%pip install transformers[sentencepiece]

In [ ]:
!python3 -m spacy download en_core_web_lg -qq
!python3 -m spacy download en_core_web_trf -qq

2023-03-09 10:46:31.438359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 10:46:32.133322: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-09 10:46:32.133391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-09 10:46:32.133402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [1]:
import json
from tqdm import tqdm
import spacy
from thinc.api import set_gpu_allocator, require_gpu
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import roc_auc_score, jaccard_score

with open('selected_tags.json', 'r') as openfile:
    selected_tags = json.load(openfile)


2023-03-21 15:44:08.466311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 15:44:10.098792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-21 15:44:10.099163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-21 15:44:10.099313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [2]:
import pandas as pd
sample_train = pd.read_csv(
    './train.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_test = pd.read_csv(
    './test.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_validation = pd.read_csv(
    './validation.csv', converters={'tag_set': eval, 'tag_list': eval})


In [8]:


# Use the GPU, with memory allocations directed via PyTorch.
# This prevents out-of-memory errors that would otherwise occur from competing
# memory pools.
set_gpu_allocator("pytorch")
require_gpu(0)


def preprocess(texts, nlp):
    removal = ['ADV', 'PRON', 'CCONJ', 'PUNCT',
               'PART', 'DET', 'ADP', 'SPACE', 'NUM', 'SYM']
    tokens = []
    cleaned_texts = []
    print("preprocessing")
    for summary in tqdm(nlp.pipe(texts, disable=["tok2vec"]), total=len(texts)):
        question_tokens = []
        for token in summary:
            if token.pos_ not in removal and not token.is_stop and token.is_alpha:
                question_tokens.append(token.lemma_)
        cleaned_texts.append(" ".join(question_tokens))
    # question_tokens = [token.lemma_ for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
    return cleaned_texts

def preprocess_transformers(texts, nlp):
    tokens = []
    removal = [ 'PUNCT', 'SPACE', 'NUM', 'SYM']
    cleaned_texts = []
    print("preprocessing")
    for summary in tqdm(nlp.pipe(texts, disable=["transformer", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"]), total=len(texts)):
        question_tokens = []
        for token in summary:
            if token.pos_ not in removal and token.is_alpha and len(question_tokens)<512:
                question_tokens.append(token.lower_)
        cleaned_texts.append(" ".join(question_tokens))
    # question_tokens = [token.lemma_ for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
    return cleaned_texts


def convert(data, outfile, nlp):
    db = spacy.tokens.DocBin()
    docs = []
    print("converting")
    for doc, labels in tqdm(nlp.pipe(data.values, as_tuples=True), total=len(data)):
        for tag in selected_tags:
            doc.cats[tag] = tag in labels
        db.add(doc)
    db.to_disk(outfile)

def evaluate_predictions(pred, mlb):
    print("Roc auc for each tag:")
    print(list(zip(mlb.classes_, roc_auc_score(y, pred, average=None))))
    print(
        f"Text roc auc macro average: {roc_auc_score(y, pred, average='macro')}")
    print(
        f"Jaccard score sample average: {jaccard_score(y, pred, average='samples')}")


### BOW

In [ ]:
nlp = spacy.load('en_core_web_lg')

#### Text

In [ ]:
train_df = sample_train[~sample_train.text.isna()].loc[:]
train_df["text_processed"] = preprocess(train_df.text, nlp)
convert(train_df.loc[:, ["text_processed", "tag_list"]],
        'text_train_bow.spacy', nlp)
test_df = sample_test[~sample_test.text.isna()].loc[:]
test_df["text_processed"] = preprocess(test_df.text, nlp)
convert(test_df.loc[:, ["text_processed", "tag_list"]],
        'text_test_bow.spacy', nlp)
validation_df = sample_validation[~sample_validation.text.isna()].loc[:]
validation_df["text_processed"] = preprocess(
    validation_df.text, nlp)
convert(validation_df.loc[:, ["text_processed", "tag_list"]],
        'text_validation_bow.spacy', nlp)


preprocessing


  0%|          | 0/105710 [00:00<?, ?it/s]

converting


  0%|          | 0/105710 [00:00<?, ?it/s]

preprocessing


...

  0%|          | 0/21648 [00:00<?, ?it/s]

preprocessing


  0%|          | 0/14149 [00:00<?, ?it/s]

converting


  0%|          | 0/14149 [00:00<?, ?it/s]

In [ ]:
!python3 -m spacy train config_BOW.cfg --output ./output_BOW --paths.train ./text_train_bow.spacy --paths.dev ./text_test_bow.spacy --gpu-id 0

2023-03-17 12:18:13.716779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 12:18:15.191947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 12:18:15.192320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 12:18:15.192476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

#### Code

In [ ]:
train_df = sample_train[~sample_train.code.isna()].loc[:]
train_df["code_processed"] = preprocess(train_df.code, nlp)
convert(train_df.loc[:, ["code_processed", "tag_list"]],
        'code_train_bow.spacy', nlp)
test_df = sample_test[~sample_test.code.isna()].loc[:]
test_df["code_processed"] = preprocess(test_df.code, nlp)
convert(test_df.loc[:, ["code_processed", "tag_list"]],
        'code_test_bow.spacy', nlp)
validation_df = sample_validation[~sample_validation.code.isna()].loc[:]
validation_df["code_processed"] = preprocess(
    validation_df.code, nlp)
convert(validation_df.loc[:, ["code_processed", "tag_list"]],
        'code_validation_bow.spacy', nlp)


preprocessing


  0%|          | 0/81619 [00:00<?, ?it/s]

converting


  0%|          | 0/81619 [00:00<?, ?it/s]

preprocessing


...

  0%|          | 0/16800 [00:00<?, ?it/s]

preprocessing


  0%|          | 0/10956 [00:00<?, ?it/s]

converting


  0%|          | 0/10956 [00:00<?, ?it/s]

In [ ]:
!python3 -m spacy train config_BOW.cfg --output ./output_code_BOW --paths.train ./code_train_bow.spacy --paths.dev ./code_test_bow.spacy --gpu-id 0

2023-03-17 13:35:08.530678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 13:35:10.172390: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 13:35:10.172770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 13:35:10.172917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

### Tok2Vec

#### Text

In [ ]:
!python3 -m spacy train config_tok2vec.cfg --output ./output_tok2vec --paths.train ./text_train_bow.spacy --paths.dev ./text_test_bow.spacy --gpu-id 0

2023-03-17 12:20:20.144707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 12:20:21.668324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 12:20:21.671812: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 12:20:21.671976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

#### Code

In [ ]:
!python3 -m spacy train config_tok2vec.cfg --output ./output_code_tok2vec --paths.train ./code_train_bow.spacy --paths.dev ./code_test_bow.spacy --gpu-id 0

2023-03-17 13:34:36.026104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 13:34:37.612346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 13:34:37.612711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 13:34:37.612871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

### RoBertA


In [23]:
nlp = spacy.load("en_core_web_trf")

loading configuration file /tmp/tmp0pknapxh/config.json
Model config RobertaConfig {
  "_name_or_path": "/tmp/tmp0pknapxh/config.json",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json



#### Text

In [ ]:
train_df = sample_train[~sample_train.text.isna()].loc[:]
train_df["text_processed"] = preprocess_transformers(train_df.text, nlp)
test_df = sample_test[~sample_test.text.isna()].sample(
    round(len(sample_test)/4))
test_df["text_processed"] = preprocess_transformers(
    test_df.text, nlp)
validation_df = sample_validation[~sample_validation.text.isna()].loc[:]
validation_df["text_processed"] = preprocess_transformers(
    validation_df.text, nlp)
convert(train_df.loc[:, ["text_processed", "tag_list"]],
        'text_train_transformer.spacy', nlp)
convert(test_df.loc[:, ["text_processed",
        "tag_list"]], 'text_test_transformer.spacy', nlp)
convert(validation_df.loc[:, [
        "text_processed", "tag_list"]], 'text_validation_transformer.spacy', nlp)


preprocessing


100%|██████████| 5322/5322 [01:00<00:00, 87.44it/s] 


In [ ]:
!python3 -m spacy train config_transformer_filled.cfg --output ./output_transformer --paths.train ./text_train_transformer.spacy --paths.dev ./text_test_transformer.spacy --gpu-id 0

2023-03-16 16:03:36.530246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 16:03:38.978276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 16:03:38.978632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 16:03:38.978786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

#### Code

In [ ]:
train_df = sample_train[~sample_train.code.isna()].loc[:]
train_df["code_processed"] = preprocess_transformers(train_df.code, nlp)
test_df = sample_test[~sample_test.code.isna()].sample(
    round(len(sample_test)/4))
test_df["code_processed"] = preprocess(test_df.code, nlp)
validation_df = sample_validation[~sample_validation.code.isna()].loc[:]
validation_df["code_processed"] = preprocess(
    validation_df.code, nlp)
convert(train_df.loc[:, ["code_processed", "tag_list"]],
        'code_train_transformer.spacy', nlp)
convert(test_df.loc[:, ["code_processed",
        "tag_list"]], 'code_test_transformer.spacy', nlp)
convert(validation_df.loc[:, [
        "code_processed", "tag_list"]], 'code_validation_transformer.spacy', nlp)


preprocessing


100%|██████████| 76005/76005 [09:12<00:00, 137.64it/s]


In [ ]:
!python3 -m spacy train config_transformer_filled.cfg --output ./output_code_transformer --paths.train ./code_train_transformer.spacy --paths.dev ./code_test_transformer.spacy --gpu-id 0

2023-03-17 09:46:03.044001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 09:46:04.556747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 09:46:04.557120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 09:46:04.557277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

### USE

In [24]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
import os
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Bert
import transformers
from transformers import *

os.environ["TF_KERAS"]='1'

In [25]:
print(tf.__version__)
print(tensorflow.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

2.11.0
2.11.0
Num GPUs Available:  1
True


In [26]:
train_df = sample_train.loc[:]
train_df.fillna("", inplace=True)
train_df["text_processed"] = preprocess_transformers(train_df.text, nlp)
train_df["code_processed"] = preprocess_transformers(train_df.code, nlp)


preprocessing


100%|██████████| 105715/105715 [00:42<00:00, 2502.29it/s]


preprocessing


100%|██████████| 105715/105715 [02:21<00:00, 749.61it/s]


In [27]:
import tensorflow_hub as hub
import torch
import gc
del nlp
gc.collect()
torch.cuda.empty_cache()

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [45]:
def feature_USE_fct(sentences, b_size) :
    batch_size = b_size
    # time1 = time.time()
    
    for step in tqdm(range(len(sentences)//batch_size)) :
        idx = step*batch_size
        feat = embed(sentences[idx:idx+batch_size])

        if step ==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))

    # time2 = np.round(time.time() - time1,0)
    return features

In [57]:
train_df_text = train_df[train_df.text != ""]
train_df_code = train_df[train_df.code != ""]


In [58]:
batch_size = 10
text_embedded_use = feature_USE_fct(train_df_text["text_processed"].to_list(), batch_size)
code_embedded_use = feature_USE_fct(
    train_df_code["code_processed"].to_list(), batch_size)


100%|██████████| 8161/8161 [02:53<00:00, 47.00it/s] 


In [59]:
from sklearn.preprocessing import MultiLabelBinarizer

sample_train.tag_list = sample_train.tag_list.apply(
    lambda tag_list: [tag for tag in tag_list if tag in selected_tags])
sample_test.tag_list = sample_test.tag_list.apply(
    lambda tag_list: [tag for tag in tag_list if tag in selected_tags])
sample_validation.tag_list = sample_validation.tag_list.apply(
    lambda tag_list: [tag for tag in tag_list if tag in selected_tags])

mlb = MultiLabelBinarizer()
mlb.fit(sample_train.tag_list.values)
y_train_text = mlb.transform(train_df_text.tag_list.values)
y_train_text_sets = train_df_text.tag_list.apply(set)
y_train_code = mlb.transform(train_df_code.tag_list.values)
y_train_code_sets = train_df_code.tag_list.apply(set)


In [60]:
from sklearn.datasets import make_multilabel_classification
import numpy as np
from xgboost import XGBClassifier

text_USE_xgb = XGBClassifier(tree_method="hist")
text_USE_xgb.fit(text_embedded_use, y_train_text)

code_USE_xgb = XGBClassifier(tree_method="hist")
code_USE_xgb.fit(code_embedded_use, y_train_code)


XGBoostError: [18:56:02] ../src/data/data.cc:455: Check failed: this->labels.Size() % this->num_row_ == 0 (144 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/aurelien/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x1fcf49) [0x7f4df63fcf49]
  [bt] (1) /home/aurelien/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x209dd2) [0x7f4df6409dd2]
  [bt] (2) /home/aurelien/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x20bab0) [0x7f4df640bab0]
  [bt] (3) /home/aurelien/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xad) [0x7f4df632997d]
  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7f5380bfae2e]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7f5380bf7493]
  [bt] (6) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0xa59d) [0x7f53802e859d]
  [bt] (7) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0x9c2d) [0x7f538

In [ ]:
val_df_text = train_df[sample_validation.text != ""]
val_df_code = train_df[sample_validation.code != ""]
y_val_text = mlb.transform(val_df_text.tag_list.values)
y_val_text_sets = val_df_text.tag_list.apply(set)
y_val_code = mlb.transform(val_df_text.tag_list.values)
y_val_code_sets = val_df_text.tag_list.apply(set)
y_pred = clf.predict(text_val_tf)
print("Roc auc for each tag:")
print(list(zip(mlb.classes_, roc_auc_score(y_val, y_pred, average=None))))
print(
    f"Text roc auc macro average: {roc_auc_score(y_val, y_pred, average='macro')}")
print(
    f"Jaccard score sample average: {jaccard_score(y_val, y_pred, average='samples')}")


## Evaluation

In [5]:




sample_validation = pd.read_csv(
    './validation.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_validation.text.fillna("", inplace=True)
sample_validation.code.fillna("", inplace=True)
sample_validation.tag_list = sample_validation.tag_list.apply(
    lambda tag_list: [tag for tag in tag_list if tag in selected_tags])

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(sample_validation.tag_list.values)
y_sets = sample_validation.tag_list.apply(set)


preprocessing


100%|██████████| 14152/14152 [01:45<00:00, 134.51it/s]


preprocessing


100%|██████████| 14152/14152 [01:40<00:00, 141.46it/s]


In [ ]:

nlp_trf = spacy.load("en_core_web_trf")
val_trf = sample_validation.loc[:]
val_trf["texts_processed"] = preprocess_transformers(val_trf.text, nlp_trf)
val_trf['codes_processed'] = preprocess_transformers(val_trf.code, nlp_trf)

nlp_text = spacy.load("./output_transformer/model-best")
nlp_code = spacy.load("./output_code_transformer/model-best")
text_cats = []
code_cats = []
for summary in tqdm(nlp_text.pipe(val_trf["texts_processed"].values), total=len(val_trf)):
    text_cats.append(summary.cats)

for summary in tqdm(nlp_code.pipe(val_trf["codes_processed"].values), total=len(val_trf)):
    code_cats.append(summary.cats)

val_trf["text_cats"] = text_cats
val_trf[val_trf.text == ""].text_cats = dict.fromkeys(selected_tags, 0)
val_trf["code_cats"] = code_cats
val_trf[val_trf.code == ""].code_cats = dict.fromkeys(selected_tags, 0)


In [68]:
THRESHOLD = 0.5
y_pred_text = [[x for x in selected_tags if y[x] > THRESHOLD]
               for y in val_trf.text_cats.values]
y_pred_code = [[x for x in selected_tags if y[x] > THRESHOLD]
               for y in val_trf.code_cats.values]

y_union = [list(set(y_pred_text[i]) | set(y_pred_code[i])) for i in range(len(y_pred_text))]

y_pred_text_t = mlb.transform(y_pred_text)
y_pred_code_t = mlb.transform(y_pred_code)
y_union_t = mlb.transform(y_union)

print("TEXT ONLY")
evaluate_predictions(y_pred_text_t, mlb)
print("CODE ONLY")
evaluate_predictions(y_pred_code_t, mlb)
print("UNION")
evaluate_predictions(y_union_t, mlb)


TEXT ONLY
Roc auc for each tag:
[('.net', 0.7692095799440233), ('android', 0.9080808690792053), ('c', 0.8447096161377827), ('c#', 0.8388032570978465), ('c++', 0.7881483148459298), ('css', 0.913996074936601), ('html', 0.8416098388612389), ('ios', 0.8734087727851785), ('iphone', 0.7788454730954467), ('java', 0.7919521783959212), ('javascript', 0.8453633686623323), ('jquery', 0.8498724053753448), ('node.js', 0.8705711658530503), ('objective-c', 0.793235451364751), ('php', 0.8618515050250596), ('python', 0.9013762832571274)]
Text roc auc macro average: 0.8419396346698025
Jaccard score sample average: 0.6432989447899001
CODE ONLY
Roc auc for each tag:
[('.net', 0.7118275048578196), ('android', 0.7014331326810528), ('c', 0.7336554822753112), ('c#', 0.7457083116089966), ('c++', 0.6881793645604458), ('css', 0.7673096924660345), ('html', 0.7189115131655768), ('ios', 0.7489205401739829), ('iphone', 0.7016473996089918), ('java', 0.6927924005318513), ('javascript', 0.7024452194357037), ('jquery', 

In [69]:

nlp_lg = spacy.load('en_core_web_lg')
val_bow = sample_validation.loc[:]
val_bow["texts_processed"] = preprocess(val_bow.text, nlp_lg)
val_bow['codes_processed'] = preprocess(val_bow.code, nlp_lg)

nlp_text = spacy.load("./output_BOW/model-best")
nlp_code = spacy.load("./output_code_BOW/model-best")
text_cats = []
code_cats = []
for summary in tqdm(nlp_text.pipe(val_bow["texts_processed"].values), total=len(val_bow)):
    text_cats.append(summary.cats)

for summary in tqdm(nlp_code.pipe(val_bow["codes_processed"].values), total=len(val_bow)):
    code_cats.append(summary.cats)

val_bow["text_cats"] = text_cats
val_bow[val_bow.text == ""].text_cats = dict.fromkeys(selected_tags, 0)
val_bow["code_cats"] = code_cats
val_bow[val_bow.code == ""].code_cats = dict.fromkeys(selected_tags, 0)


preprocessing


100%|██████████| 14152/14152 [01:15<00:00, 187.49it/s]


preprocessing


100%|██████████| 14152/14152 [00:06<00:00, 2283.60it/s]
/tmp/ipykernel_30482/610484677.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_bow[val_bow.text == ""].text_cats = dict.fromkeys(selected_tags, 0)
/tmp/ipykernel_30482/610484677.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_bow[val_bow.code == ""].code_cats = dict.fromkeys(selected_tags, 0)


In [71]:
THRESHOLD = 0.5
y_pred_text = [[x for x in selected_tags if y[x] > THRESHOLD]
               for y in val_bow.text_cats.values]
y_pred_code = [[x for x in selected_tags if y[x] > THRESHOLD]
               for y in val_bow.code_cats.values]

y_union = [list(set(y_pred_text[i]) | set(y_pred_code[i])) for i in range(len(y_pred_text))]

y_pred_text_t = mlb.transform(y_pred_text)
y_pred_code_t = mlb.transform(y_pred_code)
y_union_t = mlb.transform(y_union)

print("TEXT ONLY")
evaluate_predictions(y_pred_text_t, mlb)
print("CODE ONLY")
evaluate_predictions(y_pred_code_t, mlb)
print("UNION")
evaluate_predictions(y_union_t, mlb)


TEXT ONLY
Roc auc for each tag:
[('.net', 0.6488900967782152), ('android', 0.8301444824905723), ('c', 0.7419467792574461), ('c#', 0.7319272956228179), ('c++', 0.7019025172772958), ('css', 0.8399595601095461), ('html', 0.736338138683504), ('ios', 0.7948700679334377), ('iphone', 0.6635471627376747), ('java', 0.7256070039936248), ('javascript', 0.744058014673224), ('jquery', 0.7819770185228876), ('node.js', 0.776855033231991), ('objective-c', 0.646231952979652), ('php', 0.7956970717251355), ('python', 0.8240978956325864)]
Text roc auc macro average: 0.7490031307281007
Jaccard score sample average: 0.4823629169022046
CODE ONLY
Roc auc for each tag:
[('.net', 0.6013185899615637), ('android', 0.7079430036168805), ('c', 0.67738271317357), ('c#', 0.7018641191800484), ('c++', 0.6596744189582685), ('css', 0.7413250204306504), ('html', 0.6589245157521024), ('ios', 0.6677736187345674), ('iphone', 0.5571416184495195), ('java', 0.6612724150810498), ('javascript', 0.684897964392756), ('jquery', 0.636

In [72]:
val_tok2vec = val_bow.loc[:]
nlp_text = spacy.load("./output_tok2vec/model-best")
nlp_code = spacy.load("./output_code_tok2vec/model-best")
text_cats = []
code_cats = []
for summary in tqdm(nlp_text.pipe(val_tok2vec["texts_processed"].values), total=len(val_tok2vec)):
    text_cats.append(summary.cats)

for summary in tqdm(nlp_code.pipe(val_tok2vec["codes_processed"].values), total=len(val_tok2vec)):
    code_cats.append(summary.cats)

val_tok2vec["text_cats"] = text_cats
val_tok2vec[val_tok2vec.text == ""].text_cats = dict.fromkeys(selected_tags, 0)
val_tok2vec["code_cats"] = code_cats
val_tok2vec[val_tok2vec.code == ""].code_cats = dict.fromkeys(selected_tags, 0)


100%|██████████| 14152/14152 [00:07<00:00, 1949.08it/s]
/tmp/ipykernel_30482/308916592.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_tok2vec[val_tok2vec.text == ""].text_cats = dict.fromkeys(selected_tags, 0)
/tmp/ipykernel_30482/308916592.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_tok2vec[val_tok2vec.code == ""].code_cats = dict.fromkeys(selected_tags, 0)


In [73]:
THRESHOLD = 0.5
y_pred_text = [[x for x in selected_tags if y[x] > THRESHOLD]
               for y in val_tok2vec.text_cats.values]
y_pred_code = [[x for x in selected_tags if y[x] > THRESHOLD]
               for y in val_tok2vec.code_cats.values]

y_union = [list(set(y_pred_text[i]) | set(y_pred_code[i])) for i in range(len(y_pred_text))]

y_pred_text_t = mlb.transform(y_pred_text)
y_pred_code_t = mlb.transform(y_pred_code)
y_union_t = mlb.transform(y_union)

print("TEXT ONLY")
evaluate_predictions(y_pred_text_t, mlb)
print("CODE ONLY")
evaluate_predictions(y_pred_code_t, mlb)
print("UNION")
evaluate_predictions(y_union_t, mlb)


TEXT ONLY
Roc auc for each tag:
[('.net', 0.6421738546586514), ('android', 0.8807337192195761), ('c', 0.7965499808922081), ('c#', 0.7380627650712426), ('c++', 0.718745238765006), ('css', 0.8528525867491494), ('html', 0.792017609560249), ('ios', 0.7830064177346702), ('iphone', 0.665392220285253), ('java', 0.771801953182504), ('javascript', 0.8119293888438004), ('jquery', 0.832020834035505), ('node.js', 0.8196238383189313), ('objective-c', 0.6038419139093959), ('php', 0.8302391235253427), ('python', 0.8801803381366468)]
Text roc auc macro average: 0.7761982364305082
Jaccard score sample average: 0.5327480214810627
CODE ONLY
Roc auc for each tag:
[('.net', 0.6002668716090207), ('android', 0.7431834972600363), ('c', 0.7388924816042166), ('c#', 0.7260139695923068), ('c++', 0.697964738350655), ('css', 0.807689796929288), ('html', 0.7150599624483918), ('ios', 0.7110989711456355), ('iphone', 0.6276259641200366), ('java', 0.7048169043203596), ('javascript', 0.7214736670564063), ('jquery', 0.675